[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gabays/32M7131/blob/main/Cours_03/Cours03.ipynb)

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licence Creative Commons" style="border-width:0;float:right;\" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>

Distant Reading 2: linguistique computationnelle

# **Lemmatiser un texte**

Simon Gabay



🚨 Pour les entraînements, on va avoir besoin d'un GPU à notre meilleur ami Google: vous pouvez en demander en allant sur dans le menu en haut à gauche et en choisissant `Execution` > `Modifier le type d'exécution` puis en choisissant `GPU`.

⚠️ <font color='red'>Attention! l'usage des GPU est limité!!!! Il faut les utiliser avec parcimonie, sinon il faut payer!! Ou bien vous pouvez tout faire en ligne de commande sur les clusters HPC de l'uni.</font> Pour rappel, une documentation est disponible [ici](https://github.com/FoNDUE-HTR/Documentation/blob/master/CLUSTERS.md), mais concerne l'entraînement de modèles d'OCR.



## 1. Annoter

### 1.1 Premier test

Il nous faut d'abord installer [_pie-extended_](https://pypi.org/project/pie-extended/) pour utiliser le lemmatiseur [_Pie_](https://github.com/emanjavacas/pie). Il existe bien d'autres outils: l'un des plus populaires est [_spaCy_](https://spacy.io), qui couvre les principales langues (allemand, français, anglais, espagnol, anglais…), mais nous nous intéressons à des langues moins bien dotées et d'une nature différente (pré-orthographiques) qui nécessitent des solutions spécifiques.

In [ ]:
!pip install pie-extended

Nous téléchargeons les modèles pour le français (dit `fr`) -- une liste des modèles disponibles est [en ligne](https://pypi.org/project/pie-extended).

In [ ]:
!pie-extended download fr

Nous créons un fichier `essai.txt` un avec la phrase _je m'appelle Simon_:

In [ ]:
!echo "je m'appelle Simon" >> essai.txt

Avec `pie extended` j'annote (`tag`) avec le modèle `fr`:

In [ ]:
!pie-extended tag fr /content/essai.txt

Il me crée automatiquement un fichier du même nom en ajoutant `-pie` au nom avant l'extension. Je peux regarder le résultat en ouvrant le fichier avec la commande bash `cat`:

In [ ]:
!cat /content/essai-pie.txt

### 1.2 Un test plus ambitieux

Nous recommençons avec un texte plus complexe: la première fable de _L’École des femmes_ de Molière [disponible sur wikisource]([Wikisource](https://fr.wikisource.org/wiki/L%E2%80%99%C3%89cole_des_femmes/%C3%89dition_Chasles,_1888):

In [ ]:
!curl https://raw.githubusercontent.com/gabays/32M7131/master/Cours_03/docs/moliere.txt > /content/moliere.txt
!head -10 /content/moliere.txt

J'annote ce fichier:

In [ ]:
!pie-extended tag fr /content/moliere.txt
!head -10 /content/moliere-pie.txt

### 1.3 Un essai en latin
 Recommençons avec un texte en latin. Il faut d'abord charger le modèle latin et les _addons_

In [ ]:
!pie-extended download lasla
!pie-extended install-addons lasla

Téléchargeons désormais un texte, le _Pro Quinctio_ de Cicéron disponible sur [_The Latin Library_](https://www.thelatinlibrary.com/cicero/quinc.shtml).

In [ ]:
!curl https://raw.githubusercontent.com/gabays/32M7131/master/Cours_03/docs/Cicero_%20Pro_Quinctio.txt > /content/Cicero_%20Pro_Quinctio.txt
!head -10 /content/Cicero_%20Pro_Quinctio.txt

On annote et on observe le résultat:

In [ ]:
!pie-extended tag lasla /content/Cicero_%20Pro_Quinctio.txt
!head -10 /content/Cicero_%20Pro_Quinctio-pie.txt

## 2. Entraîner un modèle
### 2.1 _Split_ pour l'entraînement

Nous allons d'abord faire un _split_ et créer les trois jeux de données nécessaires avec [_Protogenie_](https://github.com/hipster-philology/protogenie).

Récupérons d'abord un jeu de données.

In [ ]:
!curl https://raw.githubusercontent.com/gabays/32M7131/main/Cours_03/train/lemma_Empty.txt > /content/lemma_Empty.txt
!head -20 /content/lemma_Empty.txt

Ensuite nous installons _Protogenie_.

In [ ]:
!pip install protogenie

On charge un fichier de config (pour apprendre à les fabriquer, il existe une [documentation](https://github.com/hipster-philology/protogenie/blob/master/DOCUMENTATION.md)).



In [ ]:
!curl https://raw.githubusercontent.com/gabays/32M7131/main/Cours_03/train/lemma_Empty.xml > /content/lemma_Empty.xml
!cat /content/lemma_Empty.xml

On fait le split -- ici 10% en `dev`, 10% en `eval` (ou `test`) et 80% en `train`, mais cette répartition peut changer en fonction de la quantité de données dont on dispose.

In [ ]:
!protogenie build /content/lemma_Empty.xml -d 0.1 -e 0.1 -t 0.8

### 2.2 Entrainement
On va entraîner avec [_Pie_](https://github.com/emanjavacas/pie).

In [ ]:
!pip install nlp-pie

On a besoin de renommer les fichiers créés avec _Protogénie_ en `.tsv`

In [8]:
!mv /content/output/dev/lemma_Empty.txt /content/output/dev/lemma_Empty.tsv
!mv /content/output/test/lemma_Empty.txt /content/output/test/lemma_Empty.tsv
!mv /content/output/train/lemma_Empty.txt /content/output/train/lemma_Empty.tsv

Nous avons aussi besoin d'un fichier de configuration

In [ ]:
!curl https://raw.githubusercontent.com/gabays/32M7131/main/Cours_03/train/config.json > /content/config.json
!cat /content/config.json

Ce fichier contient toutes les informations nécessaires pour l'entraînement. Le paramétrage via le fichier de configuration est extrêmement complexe, et il est difficile (impossible?) de connaître à l'avance les meilleurs choix à opérer: il faut souvent faire de nombreux tests.

Si vous voulez un peu comprendre le contenu, voici quelques explications:
```json
{
  // * General
  // model name to be used for saving
  "modelname": "latest-fro",
  // model path to be used for saving
  "modelpath": "./",
  // run test (no serialization)
  "run_test": false,
  // max length of sentences (longer sentence will be split)
  "max_sent_len": 35,
  // maximum number of sentences to process
  "max_sents": 1000000,
  // * Data
  // path or unix-like expression to file(s)/dir with training data:
  // e.g. "datasets/capitula_classic/train0.tsv""
  "input_path": "corpus_for_train/train/CORPUS.tsv",
  // path to test set (same format as input_path)
  "test_path": "corpus_for_train/test/CORPUS.tsv",
  // path to dev set (same format as input_path)
  "dev_path": "corpus_for_train/dev/CORPUS.tsv",
  // data to use as reference for breaking lines (e.g. "pos")
  "breakline_ref": "POS",
  // needed to decide for a sentence boundary (e.g. "$.")
  "breakline_data": "INUTILE",
  // maximum vocabulary size
  "char_max_size": 500,
  // maximum vocabulary size for word input
  "word_max_size": 20000,
  // min freq per item to be incorporated in the vocabulary (only used if *_max_size is 0)
  "char_min_freq": 1,
  "word_min_freq": 1,
  // char-level encoding
  "char_eos": true,
  "char_bos": true,
  // tab-format only:
  "header": true,
  // separator for csv-like files
  "sep": "\t",
  // task-related config
  "tasks": [
    // each task's name refers to the corresponding data field
    // this behaviour can be changed in case the name differs from the data field
    // by using a "target" key that refers to the target data field
    // e.g. {"name": "lemma-char", "settings": {"target": "lemma"}}
    // e.g. {"name": "lemma-word", "settings": {"target": "lemma"}}
    {
      "name": "lemma",
      "target": true,
      "context": "sentence",
      "level": "char",
      "decoder": "attentional",
      "settings": {
        "bos": true,
        "eos": true,
        "lower": true,
        "target": "lemma"
      },
      "layer": -1
    },
    {"name": "POS"}
  ],
  "task_defaults": {
    "level": "token",
    "layer": -1,
    "decoder": "linear",
    "context": "sentence"
  },
  // general task schedule params (can be overwritten in the "settings" entry of each)
  "patience": 1000000, // default to very large value
  "factor": 1, // decrease the loss weight by this amount (0, 1)
  "threshold": 0, // minimum decrease in loss to be considered an improvement
  "min_weight": 0, // minimum value a task weight can be decreased to

  // whether to include autoregressive loss
  "include_lm": true,
  // whether to share the output layer for both fwd and bwd lm
  "lm_shared_softmax": false,
  // lm settings in case it's included as an extra task
  "lm_schedule": {
    "patience": 100, "factor": 0.5, "weight": 0.2, "mode": "min"
  }
}
  ```



Et on entraine. Le nombre d'itérations (_epochs_) est volontairement limité à 10 pour ne pas gaspiller trop d'énergie inutilement. En augmentant ce nombre, on obtiendrait un bien meilleur résultat.

Vous allez voir apparaître les scores sur le `dev` our chaque itération. Pour lire les résultats, [cf. précision et rappel sur wikipedia](https://fr.wikipedia.org/wiki/Pr%C3%A9cision_et_rappel).

In [ ]:
!pie train config.json

On peut désormais utiliser ce modèle avec _Pie_:


In [ ]:
! pie tag /content/mymodel.tar /content/moliere.txt

Attention, _Pie_ n'est pas _Pie extended_. Pour ajouter le modèle à ce dernier vous pouvez suivre [le tuto](https://github.com/hipster-philology/nlp-pie-taggers#add-a-model).

Il faut désormais un modèle pour la POS – mais vous avez désormais les armes pour le faire vous-même.